# Import required python package and set the Cloudant credentials
flightPredict is a helper package used to load data into RDD of LabeledPoint

In [ ]:
%matplotlib inline
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.linalg import Vectors
from numpy import array
import numpy as np
import math
!pip install --user --upgrade -e https://github.com/ibm-cds-labs/simple-data-pipe-connector-flightstats/raw/master/lib/python/flightPredict/dist/flightPredict-0.3.tar.gz
import flightPredict
flightPredict.sqlContext = sqlContext
flightPredict.cloudantHost='XXXX'
flightPredict.cloudantUserName='XXXX'
flightPredict.cloudantPassword='XXXX'
flightPredict.weatherUrl='XXXX'

# load data from training data set and print the schema

In [ ]:
dbName = "<<Your db training set here >>"
cloudantdata = flightPredict.loadDataSet(dbName,"training")
cloudantdata.cache()
cloudantdata.printSchema()
cloudantdata.count()

# Visualize classes in scatter plot based on 2 features 

In [ ]:
flightPredict.scatterPlotForFeatures(cloudantdata, \
     "departureWeather.temp","arrivalWeather.temp","Departure Airport Temp", "Arrival Airport Temp")

In [ ]:
flightPredict.scatterPlotForFeatures(cloudantdata,\
     "departureWeather.pressure","arrivalWeather.pressure","Departure Airport Pressure", "Arrival Airport Pressure")

In [ ]:
flightPredict.scatterPlotForFeatures(cloudantdata,\
 "departureWeather.wspd","arrivalWeather.wspd","Departure Airport Wind Speed", "Arrival Airport Wind Speed")

# Load the training data as an RDD of LabeledPoint

In [ ]:
computeClassification=(lambda x:x[0])
trainingData = flightPredict.loadLabeledDataRDD("training")
print(trainingData)

# Train multiple classification models

In [ ]:
from pyspark.mllib.classification import LogisticRegressionWithLBFGS
logRegModel = LogisticRegressionWithLBFGS.train(trainingData.map(lambda lp: LabeledPoint(lp.label,\
      np.fromiter(map(lambda x: 0.0 if np.isnan(x) else x,lp.features.toArray()),dtype=np.double )))\
      , iterations=100, validateData=False, intercept=True)
print(logRegModel)

In [ ]:
from pyspark.mllib.classification import NaiveBayes
#NaiveBayes requires non negative features, set them to 0 for now
modelNaiveBayes = NaiveBayes.train(trainingData.map(lambda lp: LabeledPoint(lp.label, \
                    np.fromiter(map(lambda x: x if x>0.0 else 0.0,lp.features.toArray()),dtype=np.int)\
               ))\
          )

print(modelNaiveBayes)

In [ ]:
from pyspark.mllib.tree import DecisionTree
modelDecisionTree = DecisionTree.trainClassifier(trainingData.map(lambda lp: LabeledPoint(lp.label,\
      np.fromiter(map(lambda x: 0.0 if np.isnan(x) else x,lp.features.toArray()),dtype=np.double )))\
      , numClasses=3, categoricalFeaturesInfo={})
print(modelDecisionTree)

In [ ]:
from pyspark.mllib.tree import RandomForest
modelRandomForest = RandomForest.trainClassifier(trainingData.map(lambda lp: LabeledPoint(lp.label,\
      np.fromiter(map(lambda x: 0.0 if np.isnan(x) else x,lp.features.toArray()),dtype=np.double )))\
      , numClasses=3, categoricalFeaturesInfo={},numTrees=100)
print(modelRandomForest)

# Load Blind data from Cloudant database

In [ ]:
dbTestName="<<Your db Test set here>>"
testCloudantdata = flightPredict.loadDataSet(dbTestName,"blind")
testCloudantdata.count()

In [ ]:
testData = flightPredict.loadLabeledDataRDD("blind",computeClassification)
flightPredict.runMetrics(trainingData,modelNaiveBayes,modelDecisionTree,logRegModel,modelRandomForest)

# Run the predictive model
runModel(departureAirportCode, departureDateTime, arrivalAirportCode, arrivalDateTime)  
Note: all DateTime must use UTC format

In [ ]:
from flightPredict import run
run.useModels(modelNaiveBayes,modelDecisionTree,logRegModel,modelRandomForest)
run.runModel('BOS', "2016-02-08 20:15-0500", 'LAX', "2016-01-08 22:30-0500" )